<a href="https://colab.research.google.com/github/prof-eduardo-nunes/unicamp-mineracao_dados/blob/main/Aula_YOLOv8_Dataset_Tomates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula Prática: Detecção de Tomates com YOLOv8
## Usando Dataset Personalizado (895 Imagens)

**Objetivo:** Desenvolver um sistema completo de detecção de tomates em lavoura, desde a preparação do dataset até o treinamento e inferência.

**Dataset:** 895 imagens de tomates em estufa fornecidas pelo usuário

**Aplicações Práticas:**
- Colheita automatizada
- Monitoramento de maturação
- Estimativa de produção
- Controle de qualidade

---

## Parte 1: Configuração do Ambiente

Primeiro, vamos instalar todas as bibliotecas necessárias.

In [1]:
# Instalação das bibliotecas
!pip install "ultralytics<=8.3.40" -q
!pip install roboflow -q

print("✓ Bibliotecas instaladas com sucesso!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 52.2 MB/s eta 0:00:00
✓ Bibliotecas instaladas com sucesso!


In [2]:
# Importar bibliotecas
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import shutil
import random
from pathlib import Path
from google.colab import files

print("✓ Bibliotecas importadas!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ Bibliotecas importadas!


## Parte 2: Upload e Preparação do Dataset

### 2.1. Upload do Dataset

Faça upload do arquivo `fotostomate.zip` quando solicitado.

In [ ]:
# Upload do arquivo ZIP
print("Por favor, faça upload do arquivo fotostomate.zip")
uploaded = files.upload()

# Descompactar
!unzip -q fotostomate.zip -d dataset_original
print("✓ Dataset descompactado!")

Por favor, faça upload do arquivo fotostomate.zip


### 2.2. Exploração Inicial do Dataset

In [ ]:
# Verificar estrutura do dataset
!ls -la dataset_original/

# Contar imagens
image_dir = "dataset_original/images"
images = list(Path(image_dir).glob("*.png")) + list(Path(image_dir).glob("*.jpg"))
print(f"\nTotal de imagens: {len(images)}")

### 2.3. Visualizar Amostras do Dataset

In [ ]:
# Visualizar 6 imagens aleatórias
sample_images = random.sample(images, 6)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, img_path in enumerate(sample_images):
    img = Image.open(img_path)
    axes[idx].imshow(img)
    axes[idx].axis('off')
    axes[idx].set_title(f"{img_path.name}")

plt.tight_layout()
plt.show()

print("\n📊 Observações:")
print("- Tomates em diferentes estágios de maturação")
print("- Ambiente de estufa/cultivo protegido")
print("- Variação de iluminação e ângulos")

## Parte 3: Divisão do Dataset

Vamos dividir o dataset em conjuntos de treino (70%), validação (20%) e teste (10%).

In [ ]:
def dividir_dataset(source_dir, dest_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Divide o dataset em train/val/test
    """
    # Obter lista de imagens
    source_path = Path(source_dir)
    images = list(source_path.glob("*.png")) + list(source_path.glob("*.jpg"))

    # Embaralhar
    random.seed(42)
    random.shuffle(images)

    # Calcular divisões
    total = len(images)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)

    train_images = images[:train_end]
    val_images = images[train_end:val_end]
    test_images = images[val_end:]

    # Criar estrutura de diretórios
    dest_path = Path(dest_dir)
    for split in ['train', 'val', 'test']:
        (dest_path / 'images' / split).mkdir(parents=True, exist_ok=True)
        (dest_path / 'labels' / split).mkdir(parents=True, exist_ok=True)

    # Copiar imagens
    def copiar(img_list, split):
        for img in img_list:
            dest = dest_path / 'images' / split / img.name
            shutil.copy2(img, dest)

    copiar(train_images, 'train')
    copiar(val_images, 'val')
    copiar(test_images, 'test')

    return {
        'train': len(train_images),
        'val': len(val_images),
        'test': len(test_images)
    }

# Executar divisão
stats = dividir_dataset('dataset_original/images', 'dataset_yolo')

print("✓ Dataset dividido com sucesso!")
print(f"\nEstatísticas:")
print(f"  Treino: {stats['train']} imagens")
print(f"  Validação: {stats['val']} imagens")
print(f"  Teste: {stats['test']} imagens")

## Parte 4: Anotação do Dataset

### ⚠️ IMPORTANTE: Dataset Não Anotado

O dataset fornecido **não possui anotações**. Existem três opções:

#### Opção 1: Anotação Manual com Roboflow (Recomendado)

1. Acesse [Roboflow](https://roboflow.com/)
2. Crie um projeto de Object Detection
3. Faça upload das imagens
4. Anote as imagens (desenhe bounding boxes)
5. Defina classes: `ripe` (maduro) e `unripe` (verde)
6. Exporte no formato YOLOv8

#### Opção 2: Auto-Anotação com Modelo Pré-treinado

Usar um modelo YOLO genérico para gerar anotações iniciais (requer revisão manual).

In [ ]:
# Opção 2: Auto-anotação (EXPERIMENTAL)
# Este código gera anotações preliminares que DEVEM ser revisadas

def auto_anotar(images_dir, labels_dir, conf_threshold=0.25):
    """
    Gera anotações automáticas usando modelo pré-treinado
    ATENÇÃO: Resultados devem ser revisados manualmente!
    """
    model = YOLO('yolov8n.pt')

    images_path = Path(images_dir)
    labels_path = Path(labels_dir)
    labels_path.mkdir(parents=True, exist_ok=True)

    images = list(images_path.glob("*.png")) + list(images_path.glob("*.jpg"))

    print(f"Processando {len(images)} imagens...")

    for idx, img_path in enumerate(images, 1):
        results = model.predict(source=str(img_path), conf=conf_threshold, verbose=False)

        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]

        label_path = labels_path / f"{img_path.stem}.txt"
        detections = []

        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()

                x_center = ((x1 + x2) / 2) / w
                y_center = ((y1 + y2) / 2) / h
                width = (x2 - x1) / w
                height = (y2 - y1) / h

                class_id = 0  # Temporário
                detections.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        if detections:
            with open(label_path, 'w') as f:
                f.write('\n'.join(detections))

        if idx % 100 == 0:
            print(f"Processadas: {idx}/{len(images)}")

    print("✓ Auto-anotação concluída!")
    print("⚠️  IMPORTANTE: Revise as anotações antes de treinar!")

# Descomentar para executar auto-anotação
# auto_anotar('dataset_yolo/images/train', 'dataset_yolo/labels/train')
# auto_anotar('dataset_yolo/images/val', 'dataset_yolo/labels/val')
# auto_anotar('dataset_yolo/images/test', 'dataset_yolo/labels/test')

#### Opção 3: Usar Dataset Pré-Anotado para Demonstração

Para fins didáticos, podemos usar um dataset público já anotado para demonstrar o processo de treinamento.

In [ ]:
# Opção 3: Download de dataset pré-anotado do Roboflow
from roboflow import Roboflow

# Substitua pela sua API key do Roboflow
rf = Roboflow(api_key="YOUR_API_KEY_HERE")
project = rf.workspace("moh-s15o3").project("tomato-crop-ccemz")
dataset_demo = project.version(1).download("yolov8")

print(f"Dataset de demonstração baixado em: {dataset_demo.location}")

### 4.1. Criar Arquivo data.yaml

In [ ]:
# Criar arquivo de configuração data.yaml
yaml_content = """# Dataset de Tomates

train: images/train
val: images/val
test: images/test

nc: 2
names: ['ripe', 'unripe']
"""

with open('dataset_yolo/data.yaml', 'w') as f:
    f.write(yaml_content)

print("✓ Arquivo data.yaml criado!")

## Parte 5: Treinamento do Modelo

### 5.1. Configuração do Treinamento

**Nota:** Para treinar com o dataset fornecido, você precisa primeiro anotar as imagens (Parte 4).

In [ ]:
# Inicializar modelo
model = YOLO('yolov8s.pt')  # Modelo small (bom equilíbrio)

# Treinar
results = model.train(
    data='dataset_yolo/data.yaml',  # Ou use dataset_demo.location/data.yaml
    epochs=50,
    imgsz=640,
    batch=16,
    name='tomato_detector',
    patience=10,
    save=True,
    device=0,  # GPU
    workers=2,
    project='runs/detect'
)

print("\n✓ Treinamento concluído!")

### 5.2. Visualizar Resultados do Treinamento

In [ ]:
# Mostrar gráficos de treinamento
from IPython.display import Image as IPImage

results_img = 'runs/detect/tomato_detector/results.png'
if os.path.exists(results_img):
    display(IPImage(filename=results_img, width=900))
else:
    print("Arquivo de resultados não encontrado.")

## Parte 6: Validação do Modelo

In [ ]:
# Carregar melhor modelo
best_model = YOLO('runs/detect/tomato_detector/weights/best.pt')

# Validar
metrics = best_model.val()

# Exibir métricas
print("\n📊 Métricas de Validação:")
print(f"  mAP@0.5: {metrics.box.map50:.4f}")
print(f"  mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"  Precisão: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

## Parte 7: Inferência em Novas Imagens

### 7.1. Testar no Conjunto de Teste

In [ ]:
# Função para detecção e visualização
def detectar_e_visualizar(image_path, model, conf=0.3):
    results = model.predict(source=image_path, conf=conf, save=False, show=False)
    result_img = results[0].plot()
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    return result_img, results[0]

# Testar em imagens do conjunto de teste
test_images_dir = 'dataset_yolo/images/test'
test_images = list(Path(test_images_dir).glob("*.png"))[:6]

fig, axes = plt.subplots(2, 3, figsize=(16, 12))
axes = axes.ravel()

for idx, img_path in enumerate(test_images):
    result_img, result = detectar_e_visualizar(str(img_path), best_model)

    axes[idx].imshow(result_img)
    axes[idx].axis('off')
    axes[idx].set_title(f"Detecções: {len(result.boxes)}")

plt.tight_layout()
plt.show()

### 7.2. Upload e Teste em Imagem Própria

In [ ]:
# Upload de imagem para teste
print("Faça upload de uma imagem de tomate para testar o modelo:")
uploaded_test = files.upload()

# Processar imagem
for filename in uploaded_test.keys():
    result_img, result = detectar_e_visualizar(filename, best_model, conf=0.25)

    plt.figure(figsize=(12, 8))
    plt.imshow(result_img)
    plt.axis('off')
    plt.title(f"Resultado: {len(result.boxes)} tomates detectados")
    plt.show()

    # Detalhes das detecções
    print(f"\nDetalhes das detecções em {filename}:")
    for i, box in enumerate(result.boxes, 1):
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        class_name = result.names[class_id]
        print(f"  {i}. {class_name}: {confidence:.2%}")

## Parte 8: Exportação do Modelo

Para usar o modelo em produção ou dispositivos embarcados.

In [ ]:
# Exportar para ONNX (formato universal)
best_model.export(format='onnx')

print("✓ Modelo exportado para ONNX!")
print("\nOutros formatos disponíveis:")
print("  - tflite: Para dispositivos móveis (Android/iOS)")
print("  - coreml: Para iOS")
print("  - engine: Para GPUs NVIDIA (TensorRT)")

## Parte 9: Análise de Resultados

### 9.1. Matriz de Confusão

In [ ]:
# Visualizar matriz de confusão
confusion_matrix_path = 'runs/detect/tomato_detector/confusion_matrix.png'
if os.path.exists(confusion_matrix_path):
    display(IPImage(filename=confusion_matrix_path, width=600))
else:
    print("Matriz de confusão não encontrada.")

### 9.2. Exemplos de Predições

In [ ]:
# Visualizar predições do conjunto de validação
val_batch_path = 'runs/detect/tomato_detector/val_batch0_pred.jpg'
if os.path.exists(val_batch_path):
    display(IPImage(filename=val_batch_path, width=900))
else:
    print("Imagem de predições não encontrada.")

## Conclusão

### O que aprendemos:

1. ✅ Preparar um dataset customizado para YOLOv8
2. ✅ Dividir dataset em train/val/test
3. ✅ Processo de anotação de imagens
4. ✅ Treinar modelo de detecção de objetos
5. ✅ Avaliar performance com métricas
6. ✅ Fazer inferências em novas imagens
7. ✅ Exportar modelo para produção

### Próximos Passos:

1. **Melhorar o Dataset**: Anotar todas as 895 imagens com precisão
2. **Experimentar Hiperparâmetros**: Testar diferentes learning rates, batch sizes
3. **Data Augmentation**: Aplicar transformações para melhorar generalização
4. **Modelos Maiores**: Testar YOLOv8m ou YOLOv8l para maior precisão
5. **Deploy**: Implementar em Raspberry Pi ou Jetson Nano
6. **Integração**: Conectar com sistema de robótica agrícola

### Recursos Adicionais:

- [Documentação Ultralytics](https://docs.ultralytics.com/)
- [Roboflow Tutorials](https://blog.roboflow.com/)
- [YOLOv8 GitHub](https://github.com/ultralytics/ultralytics)

---

**Desenvolvido para Agricultura de Precisão com IA** 🍅🤖